In [87]:
import pandas as pd
import numpy as np

In [88]:
df_st=pd.read_csv("students_complete.csv")
df_sc=pd.read_csv("schools_complete.csv")

nonD_st=df_st.copy()
nonD_sc=df_sc.copy()
df_sc

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [89]:
df_st.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


# District Summary

In [90]:
T_schools=df_sc["school_name"].count()
T_Students=df_st["Student ID"].count()
T_budget=df_sc["budget"].sum()
Av_math=df_st["math_score"].mean()
Av_reading=df_st["reading_score"].mean()
P_Math=df_st[df_st.math_score>=70]["math_score"].count()/T_Students
P_Reading=df_st[df_st.reading_score>=70]["reading_score"].count()/T_Students
Ov_P=(df_st[df_st.math_score>=70]["math_score"].count()+df_st[df_st.reading_score>=70]\
      ["reading_score"].count())/(2*T_Students)

print("Total # of Schools:     "+"\033[31m# ..........\033[1;m"+"\033[1;30m{:}\033[1;m".format(T_schools))
print("Total # of Students:    "+"\033[31m# ......\033[1;m"+"\033[1;30m{:,}\033[1;m".format(T_Students))
print("Total Budget:           "+"\033[31m$ ..\033[1;m"+"\033[1;30m{:,.2f}\033[1;m".format(T_budget))
print("Average Math Score:     "+"\033[31m# ..........\033[1;m"+"\033[1;30m{:,.2f}\033[1;m".format(Av_math))
print("Average Reading Score:  "+"\033[31m# ..........\033[1;m"+"\033[1;30m{:,.2f}\033[1;m".format(Av_reading))
print("% Passing Math:         "+"\033[31m% ..........\033[1;m"+"\033[1;30m{:,.2%}\033[1;m".format(P_Math))
print("% Passing Reading:      "+"\033[31m% ..........\033[1;m"+"\033[1;30m{:,.2%}\033[1;m".format(P_Reading))
print("% Overall:              "+"\033[31m% ..........\033[1;m"+"\033[1;30m{:,.2%}\033[1;m".format(Ov_P))


Total # of Schools:     # ..........15
Total # of Students:    # ......39,170
Total Budget:           $ ..24,649,428.00
Average Math Score:     # ..........78.99
Average Reading Score:  # ..........81.88
% Passing Math:         % ..........74.98%
% Passing Reading:      % ..........85.81%
% Overall:              % ..........80.39%


# School Summary

In [91]:
bins=[0,69,100]
labels=["Failed","Passed"]

df_st["Reading_FP"]=pd.cut(df_st["reading_score"],bins=bins,labels=labels)
df_st["Math_FP"]=pd.cut(df_st["math_score"],bins=bins,labels=labels)
newm_st=df_st[df_st.Math_FP=="Passed"]
newr_st=df_st[df_st.Reading_FP=="Passed"]
newm_st=(newm_st.groupby(["school_name"])["Math_FP"].count()).reset_index()
newr_st=(newr_st.groupby(["school_name"])["Reading_FP"].count()).reset_index()
new_st=pd.merge(newr_st,newm_st,on="school_name")
new_st=pd.merge(new_st,df_sc,on="school_name")
AvM=df_st.groupby(["school_name"])["math_score"].mean().reset_index()
AvR=df_st.groupby(["school_name"])["reading_score"].mean().reset_index()
AvMR=pd.merge(AvM,AvR,on="school_name")
new_st["% Passing Reading"]=new_st["Reading_FP"]/new_st["size"]
new_st["% Passing Math"]=new_st["Math_FP"]/new_st["size"]
new_st["Overall %"]=(new_st["Math_FP"]+new_st["Reading_FP"])/(2*new_st["size"])
new_st["Per Student Budget"]=new_st["budget"]/new_st["size"]
neww=new_st.drop(columns=["Reading_FP","Math_FP","School ID"],axis=1)
neww=pd.merge(neww,AvMR,on="school_name")
neww=neww[["school_name","type","size","budget","Per Student Budget",
           "math_score","reading_score","% Passing Math","% Passing Reading",
          "Overall %"]]
neww=neww.rename(columns={"school_name":"School Name","type":"School Type",
        "budget":"Total School Budget","math_score":"Avg Math Score",
        "reading_score":"Avg Reading Score","size":"Total Students"})
neww

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall %
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,0.666801,0.819333,0.743067
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.955867
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.733639
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,0.683096,0.792990,0.738043
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.952657
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.667530,0.808630,0.738080
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,0.925059,0.962529,0.943794
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.656839,0.813164,0.735002
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,0.660576,0.812224,0.736400
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,0.945946,0.959459,0.952703


# Top Performing Schools (By Passing Rate)

In [92]:
topP=neww.sort_values("Overall %",ascending=False).head(5)
topP

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall %
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.955867
12,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,0.932722,0.973089,0.952905
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,0.945946,0.959459,0.952703
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.952657
13,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,0.938677,0.965396,0.952037


# Bottom Performing Schools (By Passing Rate)

In [93]:
tailF=neww.sort_values("Overall %",ascending=False).tail(5)
tailF

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall %
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,0.683096,0.792990,0.738043
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,0.660576,0.812224,0.736400
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.656839,0.813164,0.735002
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.733639
10,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,0.663666,0.802201,0.732933


# Math Scores by Grade

In [94]:
dft=df_st.groupby(["school_name","grade"])["math_score"].mean().reset_index()
dfp=dft.pivot(index="school_name",columns="grade",values="math_score")
dfp=dfp[["9th","10th","11th","12th"]]
dfp

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [131]:
dfp.describe()

grade,9th,10th,11th,12th
count,15.000000,15.000000,15.000000,15.000000
mean,80.351617,80.378948,80.575873,80.423811
std,3.226342,3.479296,3.538353,3.372793
min,76.403037,75.908735,76.395626,76.179963
25%,77.135767,76.843944,77.027044,77.168968
50%,82.044010,82.917411,82.765560,82.855422
75%,83.342730,83.400912,83.667788,83.426603
max,83.787402,84.229064,85.000000,84.121547


# Reading Scores by Grade

In [77]:
dfG=df_st.groupby(["school_name","grade"])["reading_score"].mean().reset_index()
dff=dfG.pivot(index="school_name",columns="grade",values="reading_score")
dff=dff[["9th","10th","11th","12th"]]
dff

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [130]:
dff.describe()

grade,9th,10th,11th,12th
count,15.000000,15.000000,15.000000,15.000000
mean,82.513318,82.505439,82.559485,82.554817
std,1.408513,1.477121,1.643606,1.753843
min,80.632653,80.629808,80.403642,80.305983
25%,81.229656,81.084948,80.905227,80.884797
50%,83.369193,83.324561,83.585542,82.781671
75%,83.768062,83.759827,83.985928,84.180564
max,84.122642,84.254157,84.373786,84.698795


In [96]:
bins=[0,585,614,644,675]
labelx=["<$585","$585-$614","$615-$644","$645-$675"]
neww["$/Student"]=pd.cut(neww["Per Student Budget"],bins=bins,labels=labelx)
neww

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall %,$/Student
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,0.666801,0.819333,0.743067,$615-$644
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.955867,<$585
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.733639,$615-$644
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,0.683096,0.792990,0.738043,$615-$644
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.952657,$615-$644
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.667530,0.808630,0.738080,$645-$675
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,0.925059,0.962529,0.943794,<$585
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.656839,0.813164,0.735002,$645-$675
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,0.660576,0.812224,0.736400,$645-$675
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,0.945946,0.959459,0.952703,$585-$614


# Scores by School Spending

In [114]:
dfB=neww.groupby("$/Student")
mB=dfB["Avg Math Score"].mean().reset_index()#.set_index("Age Groups")
rB=dfB["Avg Reading Score"].mean()
rmB=pd.merge(mB,rB,on="$/Student")
pmB=dfB["% Passing Math"].mean()
prB=dfB["% Passing Reading"].mean()
prmB=pd.merge(pmB,prB,on="$/Student")
MB=pd.merge(rmB,prmB,on="$/Student")
oB=dfB["Overall %"].mean()
MB=pd.merge(MB,oB,on="$/Student")
MB

,$/Student,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall %
0,<$585,83.455399,83.933814,0.934601,0.966109,0.950355
1,$585-$614,83.599686,83.885211,0.942309,0.959003,0.950656
2,$615-$644,79.079225,81.891436,0.756682,0.861066,0.808874
3,$645-$675,76.997210,81.027843,0.661648,0.811340,0.736494


# Scores by School Size

In [126]:
binz=[0,999,1999,4999]
labelS=["(<1000) Small","(1000-2000) Medium","(2000-5000) Large"]
neww["School Size"]=pd.cut(neww["Total Students"],bins=binz,labels=labelS)
mP=neww.groupby(["School Size"])["Avg Math Score"].mean()
rP=neww.groupby(["School Size"])["Avg Reading Score"].mean()
mrP=pd.merge(mP,rP,on="School Size")
pmP=neww.groupby(["School Size"])["% Passing Math"].mean()
prP=neww.groupby(["School Size"])["% Passing Reading"].mean()
oP=neww.groupby(["School Size"])["Overall %"].mean()
mpP=pd.merge(pmP,prP,on="School Size")
mpP=pd.merge(mpP,oP,on="School Size")
mpP=pd.merge(mrP,mpP,on="School Size")
mpP

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall %
School Size,,,,,
(<1000) Small,83.821598,83.929843,0.935502,0.960994,0.948248
(1000-2000) Medium,83.374684,83.864438,0.935997,0.967907,0.951952
(2000-5000) Large,77.746417,81.344493,0.699634,0.827666,0.763650


# Scores by School Type

In [133]:
sT=neww.groupby(["School Type"])["Avg Math Score"].mean()
rT=neww.groupby(["School Type"])["Avg Reading Score"].mean()
msT=neww.groupby(["School Type"])["% Passing Math"].mean()
rsT=neww.groupby(["School Type"])["% Passing Reading"].mean()
oT=neww.groupby(["School Type"])["Overall %"].mean()
AvBud=neww.groupby(["School Type"])["Per Student Budget"].mean()
mergeT=pd.merge(sT,rT,on="School Type")
mergeT=pd.merge(mergeT,msT,on="School Type")
mergeT=pd.merge(mergeT,rsT,on="School Type")
mergeT=pd.merge(mergeT,oT,on="School Type")
mergeT=pd.merge(mergeT,AvBud,on="School Type")
mergeT

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall %,Per Student Budget
School Type,,,,,,
Charter,83.473852,83.896421,0.936208,0.965865,0.951037,599.500000
District,76.956733,80.966636,0.665485,0.807991,0.736738,643.571429


# 1. It almost looks like there is an inverse correlation between high average Overall passing grades & budget per student, this can be seen from Top Performing School & Bottom Performing School tables.
# 2. From the Reading Scores by grade it can be concluded the Reading grades through the grades in all the school, the average & median values are very close (standard deviation around 1.5), grades all between 80 - 84 points. From the Math Scores the same cannot be concluded, easiy can be verified there is a higher standard deviation of 3.5, being this subject the main cause of difference performance of the schools in the district.
# 3. Impresive the performance of the Charter's Schools compared to District Schools, specially in the Math subject. I added the column of Per Student Budget and it also shows that Charter Schools spend 7.4% less per student than the District Schools.